# Origen de datos

El origen de los datos es un proyecto de Kaggle. Una descripción se puede encontrar aquí:

https://www.kaggle.com/dcohen21/8anu-climbing-logbook

En este caso **se trata de una base de datos sqlite3** comprimida en zip. Contiene la información obtenida mediante webscraping del sitio www.8a.nu

Este sítio basicamente es un portal de escaladores en el que los usuarios pueden logarse e ir subiendo sus ascensos para compartir con otros su progresión y experiencias sobres las vías que escala.

En este notebook voy a realizar dos tareas principalmente:

- Explorar la base de datos para ver que información (tablas) proporciona.
- Generar ficheros CSV y un excel "limpios" con la información necesaria para el modelo planteado.
- Plantear las preguntas que voy a querer resolver en la práctica.
- Filtrar los ascensos a estudiar limitandonos a los que se han producido en el año 2017

***NOTA IMPORTANTE***:

**Para la correcta ejecución del codigo de este notebook, el zip de la base de datos de Kaggle debe estar en la ruta:**

**./data/8anu-climbing-logbook.zip**


# Modelo conceptual

Las entidades del modelo buscado son:

![Modelo conceptual](./img/ModeloConceptual.PNG)


# Modelo de relación

Las entidades se relacionan de la siguiente manera

![Modelo de relación](./img/ModeloRelacional.PNG)


# Preguntas a responder en la práctica

### 1.a) Los 10 escaladores (hombres) más activos.

### 1.b) Los 10 escaladoras (mujeres) más activas.

### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad

### 3. Dificultad media de los ascensos del escalador más activo

### 4.a) Los 10 ascensos mas dificiles

### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente


# Obtencion y limpieza de la información

Para explorar la información que contiene esa base de datos obtenida por webscraping sigo los siguientes pasos:

## Descompresión de la base de datos y conexión

In [1]:
import io
import os
import zipfile

dataDirectory = './data/'
zipFilePath = dataDirectory + "8anu-climbing-logbook.zip"
sqliteDataBase = dataDirectory + "database.sqlite"

if not os.path.exists(sqliteDataBase):
    print ("Descomprimiendo base de datos...")
    
    with zipfile.ZipFile(zipFilePath, "r") as zipStream:
        zipStream.extractall(dataDirectory)
    
print ("Base de datos descomprimida!")


Base de datos descomprimida!


In [2]:
import sqlite3
import pandas as pd
import numpy as np

dbConnection = sqlite3.connect(dataDirectory + 'database.sqlite')


## Identificación de tablas

Para ello conectamos con la base de datos y lanzamos la query que me dice qué tablas hay definidas en la base de datos.

In [3]:

pd.read_sql_query('''
SELECT name
FROM sqlite_master
WHERE type='table'
ORDER BY name;
''', dbConnection)




,name
0,ascent
1,grade
2,method
3,user


La base de datos tiene cuatro tablas que, por el nombre y conociendo un poco el mundillo de la escalada, parece que van a contener la siguiente información:

- **ascent**: Tabla de ascensos
- **grade**: Tabla de grados de dificultad de los ascensos
- **method**: Tabla de metodos de ascenso
- **user**: Tabla de usuarios (escaladores)

En los siguientes apartados voy a ir tabla a tabla limpiando y obteniendo los datos de la manera mas adecuada posible para respoder a las preguntas que he planteado.


##  Extraccion de datos de dificulades de los ascensos

La tabla 'grades' es una tabla de dificultades o grados. El identificador de grado va en orden ascendente con la dificultad.


In [4]:
df_dificultades = pd.read_sql_query(
    '''
    SELECT *
    FROM grade
    LIMIT 10
    ''',
    dbConnection);
list(df_dificultades)

['id',
 'score',
 'fra_routes',
 'fra_routes_input',
 'fra_routes_selector',
 'fra_boulders',
 'fra_boulders_input',
 'fra_boulders_selector',
 'usa_routes',
 'usa_routes_input',
 'usa_routes_selector',
 'usa_boulders',
 'usa_boulders_input',
 'usa_boulders_selector']

Existen mas campos en esta tabla pero los rpincipales se refieren a las escalas de graduación mas aceptadas: 

- fra_routes: la escala francesa de graduacion de vias de escalada deportiva y clásica
- usa_routes: la escala estadounidense de la graduación de vías de escalada deportiva y clásica
- usa_boulders: la escala estadounidense de la graduación de escalada en bloque (otra disciplina de la escalada) 

De esta tabla extraeré los datos para construir una tabla de equivalencia de grados de dificultad:

In [5]:
df_dificultades = pd.read_sql_query(
    '''
    SELECT id as id_dificultad,
           fra_routes as "grado_frances",
           usa_routes as "grado_usa",
           usa_boulders as "grado_bloque_usa"
    FROM grade
    ''',
    dbConnection);
df_dificultades.head(10)

,id_dificultad,grado_frances,grado_usa,grado_bloque_usa
0,1,-,3/4,VB
1,2,1,,
2,3,1a,,
3,4,1b,,
4,5,1c,,
5,6,1+,,
6,7,2,5.1,VB
7,8,2a,,
8,9,2b,,
9,10,2c,,


Vamos a comprobar que existen todos los identificadores desde el 1 hasta la longitud del data frame. Esto nos facilitará hacer calculos de grados medios, etc...

In [6]:
id_grados_tabulados = df_dificultades['id_dificultad'].unique()
id_grados_esperados = range(1,max(df_dificultades['id_dificultad']+1))
id_grados_perdidos = list(set(id_grados_esperados)^set(id_grados_tabulados));
print('Identificadores de grados desaparecidos: ',id_grados_perdidos)

('Identificadores de grados desaparecidos: ', [48, 61, 74])


Faltan estos 3 identificadores de dificultad. Como voy a hacer promedios de dificultad y redondeos en base a ese identificador, me interesa que estos registros existan. Esto es debido a que si un promedio cae sobre ellos, no podría dar un grado asociado. Por ello, voy a introducirlos con valores vacíos excepto el identificador.

In [7]:
for id_perdido in id_grados_perdidos :
    df=pd.DataFrame([[id_perdido, '', '', '']], columns=list(df_dificultades));
    df_dificultades = df_dificultades.append(df, ignore_index=True);


In [8]:
df_dificultades = df_dificultades.sort_values(['id_dificultad'],ascending=[True]);
df_dificultades = df_dificultades.reset_index(drop=True)
df_dificultades.tail()

,id_dificultad,grado_frances,grado_usa,grado_bloque_usa
81,82,9b+/9c,,
82,83,9c,,
83,84,9c/+,,
84,85,9c+,,
85,86,9c+/10a,,


Finalmente se observa que existen muchos datos vacios. Esto se debe a que las escalas de dificultad tienen solapamientos de valores. Por ello haré una sustitución hacia adelante que rellene esos datos vacios

In [9]:
df_dificultades.replace(r'^\s*$', np.nan, regex=True, inplace = True);
df_dificultades = df_dificultades.fillna(method='ffill');

df_dificultades.head(10)

,id_dificultad,grado_frances,grado_usa,grado_bloque_usa
0,1,-,3/4,VB
1,2,1,3/4,VB
2,3,1a,3/4,VB
3,4,1b,3/4,VB
4,5,1c,3/4,VB
5,6,1+,3/4,VB
6,7,2,5.1,VB
7,8,2a,5.1,VB
8,9,2b,5.1,VB
9,10,2c,5.1,VB


##  5. Extraccion de datos de tipos de encadenamiento

Esta información la extraremos de la tabla 'method' que contiene lo siguiente:

Los tipos de encadenamiento basicamente son 4 por orden de dificultad:

- **"On sight"** o **"A vista"**: el escalador, sin poseer mas información que la visualización de la via y sin haber intentado subir anteriormente por ella, asciende con exito y de primero (desplegando la cuerda que le asegura) la via, sin colgarse de ningun seguro para descansar ni sin sufrir ninguna caida.

- **"Flash"**: el escalador, sin haber intentado anteriormente la vía pero recibiendo indicaciones de un segundo escalador, asciende con exito y de primero (desplegando la cuerda que le asegura) la vía, sin colgarse ni sufrir ninguna caida

- **"Red point"**: el escalador, habiendo intentado anteriormente la vía, asciende con exito y de primero (desplegando la cuerda que le asegura) la via, sin colgarse de ningún seguro para descansar ni sin sufrir ninguna caida.

- **"Top rope"** o "de segundo": el escalado asciende la via con exito con la cuerda que le asegura instalada anteriormente por otro escalador. En este caso, no existe riesgo de caida ya que la cuerda que asegura al escalador siempre lo hace desde arriba.


In [10]:
df_tipos_encadenamiento = pd.read_sql_query(
    '''
    SELECT *
    FROM method
    ''',
    dbConnection)

df_tipos_encadenamiento.head()

,id,score,shorthand,name
0,1,0,redpoint,Redpoint
1,2,53,flash,Flash
2,3,145,onsight,Onsight
3,4,-52,toprope,Toprope
4,5,95,onsight,Onsight



**IMPORTANTE** Esta tabla muestra dos categorías repetidas (Onsight) asi que lo tendré en cuenta más adelante al obtener los datos de ascensos, para reemplazar en la clave foranea de dificultad 5 por la clave 3 que es la que voy a dejar.


In [11]:
df_tipos_encadenamiento = pd.read_sql_query(
    '''
    SELECT id as id_tipo_encadenamiento,
           name as tipo_encadenamiento
    FROM method
    ''',
    dbConnection);
df_tipos_encadenamiento = df_tipos_encadenamiento.drop([4]);
df_tipos_encadenamiento.head()

,id_tipo_encadenamiento,tipo_encadenamiento
0,1,Redpoint
1,2,Flash
2,3,Onsight
3,4,Toprope


##   Extraccion de datos de Ascensos

La tabla de ascenosos es 'ascent'. Como era de esperar, se trata de una tabla mucho más grande que las anteriores. Ire filtrando en base a un creiterio de integridad y bondad de la información, y posteriormente limitaré por fecha los ascensos a tratar.

In [12]:
pd.read_sql_query(
    '''
    SELECT count()
    FROM ascent;
    ''', 
    dbConnection)

,count()
0,4111877


Campos de la tabla

In [13]:
df_ascensos = pd.read_sql_query(
    '''
    SELECT *
    FROM ascent
    LIMIT 1
    ''',
    dbConnection);
list(df_ascensos)

['id',
 'user_id',
 'grade_id',
 'notes',
 'raw_notes',
 'method_id',
 'climb_type',
 'total_score',
 'date',
 'year',
 'last_year',
 'rec_date',
 'project_ascent_date',
 'name',
 'crag_id',
 'crag',
 'sector_id',
 'sector',
 'country',
 'comment',
 'rating',
 'description',
 'yellow_id',
 'climb_try',
 'repeat',
 'exclude_from_ranking',
 'user_recommended',
 'chipped']

De todas esas columnas me interesan aquellas que tienen bien definido los siguientes campos:

- **name** : nombre de la vía. No consideraré vías que no tienen nombre o este contiene solo simbolos de interrogación. Para minimizar efectos de la capitalización converitiré esta información a mayusculas.
- **crag** : nombre del risco o zona de escalada. Para minimizar efectos de la capitalización converitiré esta información a mayusculas.
- **sector**: nombre del sector. Para minimizar efectos de la capitalización converitiré esta información a mayusculas.
- **country**: identificador del pais en el que está la vía
- **method_id**: vamos a transformarlo de manera que evitemos la duplicidad en el tipo "On Sight que hemos comentado anteriormente.
- **date** : el campo date tiene la fecha en tiempo unix. Por claridad la convertiré a fecha


In [14]:
df_ascensos = pd.read_sql_query(
    '''
    SELECT user_id AS "id_escalador",
           grade_id AS "id_dificultad",
           CASE WHEN method_id==5 then 3
                ELSE method_id
           END AS "id_tipo_encadenamiento",
           UPPER(name) AS "nombre_via",
           UPPER(crag) AS "risco",
           UPPER(sector) AS "sector",
           strftime('%Y-%m-%d', datetime(date, 'unixepoch'))  AS "fecha",
           country AS "pais"
    FROM ascent
    WHERE name IS NOT NULL AND name!="" AND name NOT LIKE "?%"
          AND crag IS NOT NULL AND crag!=""
          AND sector IS NOT NULL AND sector!=""
          AND country IS NOT NULL AND country!=""
    ''',
    dbConnection);

df_ascensos.head()


,id_escalador,id_dificultad,id_tipo_encadenamiento,nombre_via,risco,sector,fecha,pais
0,1,36,3,THE KING AND I,RAILAY,DUM'S KITCHEN,1999-02-06,THA
1,1,36,3,MR BIG,SJöäNDA,HUVUDVäGGEN,1999-07-26,SWE
2,1,36,3,TAK SKA DU HA,SJöäNDA,HUVUDVäGGEN,1999-07-26,SWE
3,1,38,3,VALENTINE,RAILAY,MUAI THAI,1998-12-18,THA
4,1,38,1,NUAT HIN (MASSAGE THE ROCK),RAILAY,MUAI THAI,1999-01-13,THA


In [15]:
# numero de ascensos una vez filtrados los que tienen datos no deseados
len(df_ascensos.index)

2804520

Ahora filtraremos el dataframe quedandonos sólo con las ascensiones las ultimas ascensiones desde enero de 2017:

In [16]:
df_ascensos = df_ascensos[df_ascensos['fecha']>'2016-12-31'];
len(df_ascensos.index)

207599

In [17]:
df_ascensos.head()

,id_escalador,id_dificultad,id_tipo_encadenamiento,nombre_via,risco,sector,fecha,pais
172880,9340,44,1,BURDEN CHUCHEN,SIURANA,L'OLLA,2017-01-02,ESP
365806,11635,53,1,ACAMPAMENTO BASE,COCALZINHO,PISTA,2017-05-06,BRA
462997,25134,53,1,LADY DIE,HELL,Rå NYTELSE,2017-09-05,NOR
582046,19578,40,1,POQUITO A POCO,PATONES,MARACAIBO,2017-07-13,ESP
617768,21747,51,1,LES 5 SOEURS DE TERRES-NEUVES,FREYR,AL LEGNE,2017-04-02,BEL


##  Extraccion de datos de escaladores

De la tabla de usuarios obtendré los datos de la tabla de escaladores de mi modelo conceptual.

El numero de usuarios es bastante grande:

In [18]:
pd.read_sql_query(
    '''
    SELECT count()
    FROM user;
    ''', 
    dbConnection)


,count()
0,62593


Primero filtraré los campos de información para cada escalador:

In [19]:
df_escaladores = pd.read_sql_query(
    '''
    SELECT *
    FROM user
    LIMIT 1
    ''',
    dbConnection);

list(df_escaladores)

['id',
 'first_name',
 'last_name',
 'city',
 'country',
 'sex',
 'height',
 'weight',
 'started',
 'competitions',
 'occupation',
 'sponsor1',
 'sponsor2',
 'sponsor3',
 'best_area',
 'worst_area',
 'guide_area',
 'interests',
 'birth',
 'presentation',
 'deactivated',
 'anonymous']

De esta información me quedaré sólo con las columnas útiles para mi modelo:
1. id
2. first_name
3. last_name
4. city
5. country
6. sex
7. started
8. birth

Para ello hago una query tomando solo esos campos sobre toda la tabla trasformando la tabla a las siguientes columnas:

   | id | nombre | sexo | fecha_nacimiento | ciudad | pais | año_comienzo |
   | -- | ------ | ---- | ---------------- | ------ | ---- | ------------ |
   |    |        |      |                  |        |      |              |

In [20]:
df_escaladores = pd.read_sql_query(
    '''
    SELECT id as id_escalador,
           first_name || ' ' || last_name as "nombre",
           CASE WHEN sex=0 THEN 'Hombre'
                WHEN sex=1 THEN 'Mujer'
                ELSE NULL
           END as "sexo",
           birth as "fecha_nacimiento",
           city as "ciudad",
           country as "pais",
           started as "comienzo"
    FROM user
    ''',
    dbConnection);

df_escaladores.head()

,id_escalador,nombre,sexo,fecha_nacimiento,ciudad,pais,comienzo
0,1,Leif Jägerbrand,Hombre,1976-03-10,Göteborg,SWE,1996
1,2,Andreas Collisch,Hombre,None,stockholm,SWE,2000
2,3,Magnus Öberg,Hombre,1973-09-09,Umeå,SWE,1995
3,4,Annika Frodi-Lundgren (f),Mujer,1984-07-26,Goteborg,SWE,2001
4,5,Joe McLoughlin,Hombre,1969-05-07,North Attleboro,USA,1991


Es posible que muchos de estos escaldores (usuarios) en el ultimo año no hayan tenido actividad, por ello voy a filtrar de esta tabla aquellos escaladores que no han registrado ascensos en el periodo de tiempo elegido para analizar.

Primero identifico qué usuarios ha tenido actividad ese periodo de tiempo:

In [21]:
def extraeEscaladoresConAscensos(escaladores, ascensos):
    ids_escaladores_con_ascensos = df_ascensos.id_escalador.unique();
    return df_escaladores[df_escaladores.id_escalador.isin(ids_escaladores_con_ascensos)]

In [22]:
df_escaladores = extraeEscaladoresConAscensos(df_escaladores, df_ascensos)
df_escaladores.head()

,id_escalador,nombre,sexo,fecha_nacimiento,ciudad,pais,comienzo
4,5,Joe McLoughlin,Hombre,1969-05-07,North Attleboro,USA,1991
6,10,Jens Larssen,Hombre,1965-06-22,Göteborg,SWE,1992
24,28,Knut Rokne,Hombre,1972-03-27,Calgary,CAN,1988
30,35,Jason Kester,Hombre,1971-08-12,portland,USA,1992
33,38,Alan Cassidy,Hombre,1982-12-10,Glasgow,GBR,1993


In [23]:
ids_escaladores_con_ascensos = df_ascensos.id_escalador.unique();
ids_escaladores = df_escaladores.id_escalador.unique();
print ('Numero de escaladores en el data frame de escaladores', len(ids_escaladores));
print ('Numero de escaladores en el data frane de ascensos', len(ids_escaladores_con_ascensos));

('Numero de escaladores en el data frame de escaladores', 8081)
('Numero de escaladores en el data frane de ascensos', 8082)


Curiosamente, esto ultimo me indica que hay un indice de escalador que aparece en los ascensos y que no está en la lista de usuarios. A continuación identificamos el identificador del usuario perdido y vemos cuantos ascensos tiene registrado dicho usuario.

In [24]:
id_escalador_perdido = list(set(ids_escaladores)^set(ids_escaladores_con_ascensos))[0]
print ('ID de escalador perdido: ', id_escalador_perdido)
print ('Numero de ascensos del escalador perdido: ', len(df_ascensos[df_ascensos.id_escalador==id_escalador_perdido]))

('ID de escalador perdido: ', 10877)
('Numero de ascensos del escalador perdido: ', 12)


Eliminamos dichos registros de la tabla de ascensos. Son muy pocos respecto al global

In [25]:
df_ascensos = df_ascensos[df_ascensos.id_escalador!=id_escalador_perdido];

Ademas, se observa que en este data frame de escaladores pueden haber fechas de nacimiento a None y codigos de pais vacíos. Vamos a rellenar esa información:

In [26]:
# limpiamos cadenas vacias o solo con espacios en pais y fechas de nacimiento vacias
df_escaladores['pais'].replace(r'^\s*$', '---', regex=True, inplace = True);
df_escaladores['ciudad'].replace(r'^\s*$', '---', regex=True, inplace = True);
df_escaladores['fecha_nacimiento'].fillna(value='2555-12-31', inplace =True);


# Evaluacion final de data frames


Vamos a hacer una comprobación sobre el data frame de ascensos resultante conociendo la siguiente información relacionada con una de las preguntas:

Este año 2017 es la primera vez en la historia en la que se ha encadenado una via de 9c. El artífice de esa azaña es Adam Ondra, que aparece en segundo lugar en la query:

https://www.youtube.com/watch?v=wdCtHqmMnjw

https://www.youtube.com/watch?v=7Xmx2gYwsYo

Vamos a comprobar que si es asi:


In [27]:
df_ascenso_maxima_dificultad = df_ascensos[df_ascensos['id_dificultad'] == max(df_ascensos['id_dificultad'])]
df_ascenso_maxima_dificultad

,id_escalador,id_dificultad,id_tipo_encadenamiento,nombre_via,risco,sector,fecha,pais
2737952,48467,83,3,DUMMY,KALYMNOS,DUMMY,2017-07-11,GRC
2801769,1476,83,1,SILENCE,FLATANGER,HANSHALLAREN,2017-09-02,NOR


In [28]:
max_dificultad_2017 = df_ascenso_maxima_dificultad['id_dificultad'].values
df_dificultades[df_dificultades['id_dificultad'].isin(max_dificultad_2017)]

,id_dificultad,grado_frances,grado_usa,grado_bloque_usa
82,83,9c,5.15c,V20


In [29]:
max_dificultad_id_escaladores_2017 = df_ascenso_maxima_dificultad['id_escalador'].values
df_escaladores[df_escaladores['id_escalador'].isin(max_dificultad_id_escaladores_2017)]

,id_escalador,nombre,sexo,fecha_nacimiento,ciudad,pais,comienzo
1422,1476,Adam Ondra,Hombre,1993-02-05,Brno,CZE,1999
45573,48467,Kamil Kamil,Hombre,2555-12-31,London,GBR,2012


Vemos que este año se han registrado dos ascensos de 9c. El de Adam Ondra es un hecho historico, por lo que el segundo es de dudosa veracidad. De hecho es constatable también que en Grecia no existen vía graduadas con dificultad 9c. Por ello vamos a eliminar tanto al usuario como a los registros de ascensos del usuario de dudosa veracidad:

In [30]:
df_ascensos = df_ascensos[df_ascensos['id_escalador']!=48467];
df_escaladores = df_escaladores[df_escaladores['id_escalador']!=48467];

Comprobamos que ahora sólo existe un ascenso de dificultad 9c (id_dificultad=83)

In [31]:
df_ascensos[df_ascensos['id_dificultad'] == max(df_ascensos['id_dificultad'])]

,id_escalador,id_dificultad,id_tipo_encadenamiento,nombre_via,risco,sector,fecha,pais
2801769,1476,83,1,SILENCE,FLATANGER,HANSHALLAREN,2017-09-02,NOR


In [32]:
df_dificultades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
id_dificultad       86 non-null int64
grado_frances       86 non-null object
grado_usa           86 non-null object
grado_bloque_usa    86 non-null object
dtypes: int64(1), object(3)
memory usage: 2.8+ KB


In [33]:
df_tipos_encadenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 2 columns):
id_tipo_encadenamiento    4 non-null int64
tipo_encadenamiento       4 non-null object
dtypes: int64(1), object(1)
memory usage: 96.0+ bytes


In [34]:
df_ascensos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207330 entries, 172880 to 2804519
Data columns (total 8 columns):
id_escalador              207330 non-null int64
id_dificultad             207330 non-null int64
id_tipo_encadenamiento    207330 non-null int64
nombre_via                207330 non-null object
risco                     207330 non-null object
sector                    207330 non-null object
fecha                     207330 non-null object
pais                      207330 non-null object
dtypes: int64(3), object(5)
memory usage: 14.2+ MB


In [35]:
df_escaladores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8080 entries, 4 to 62592
Data columns (total 7 columns):
id_escalador        8080 non-null int64
nombre              8080 non-null object
sexo                8080 non-null object
fecha_nacimiento    8080 non-null object
ciudad              8080 non-null object
pais                8080 non-null object
comienzo            8080 non-null int64
dtypes: int64(2), object(5)
memory usage: 505.0+ KB


# Diezmado de la información

Vamos a reducir el numero de ascensos a tratar para hacerlo manejable en los ejercicios de la práctica. Para ello vamos a diezmar de la siguiente manera:

- Nos quedaremos con el 10% de los ascensos vias por debajo de 6a
- Nos quedaremos con el 5% de las ascensos entre 6a y 6c+
- Nos quedaremos con el 5% de los ascensos entre 7a y 7c+
- Nos quedaremos con el 10% de los ascensos entre 8a y 8c+
- Nos quedaremos con el 100% de lo ascensos por encima de 9a

Tras filtrar los ascensos, volveremos a filtrar los escaladores que no tienen ascensos en esa muestra


In [36]:
samples = [[0.10, -1, 36], [0.05, 36, 49], [0.05, 49, 62], [0.10, 62, 75], [1, 75, 90]]

df_ascensos_diezmado = df_ascensos.sample(frac=0)

for sample in samples:
    df_aux = df_ascensos[(df_ascensos['id_dificultad']>=sample[1]) & (df_ascensos['id_dificultad']<sample[2])];
    df_aux = df_aux.sample(frac=sample[0]);
    df_ascensos_diezmado = pd.concat([df_ascensos_diezmado, df_aux]);

print('Ascensos ANTES del diezmado',len(df_ascensos));
print('Ascensos DESPUES del diezmado',len(df_ascensos_diezmado));


('Ascensos ANTES del diezmado', 207330)
('Ascensos DESPUES del diezmado', 11832)


In [37]:
df_escaladores_diezmado = extraeEscaladoresConAscensos(df_escaladores, df_ascensos_diezmado);
print('Escaladores ANTES del diezmado',len(df_escaladores));
print('Escaladores DESPUES del diezmado',len(df_escaladores_diezmado));


('Escaladores ANTES del diezmado', 8080)
('Escaladores DESPUES del diezmado', 8080)


## Escritura de ficheros CSV

In [38]:
df_dificultades.to_csv('./data/dificultades.csv', encoding='utf-8', index=False);
df_tipos_encadenamiento.to_csv('./data/tipos_encadenamiento.csv', encoding='utf-8', index=False);

df_ascensos.to_csv('./data/ascensos_2017.csv', encoding='utf-8', index=False);
df_ascensos_diezmado.to_csv('./data/ascensos_lite_2017.csv', encoding='utf-8', index=False);

df_escaladores.to_csv('./data/escaladores_2017.csv', encoding='utf-8', index=False);
df_escaladores_diezmado.to_csv('./data/escaladores_lite_2017.csv', encoding='utf-8', index=False);
